
- Questions:
  1. What are the top 5 countries with the highest internet use (by population share)?
  2. How many people had internet access in those countries in 2019?
  3. What are the top 5 countries with the highest internet use for each of the following regions:  'Middle East & North Africa', 'Latin America & Caribbean', 'East Asia & Pacific', 'South Asia', 'North America', 'Europe & Central Asia'?
  4. Create a visualization for those five regions' internet usage over time.
  5. What are the 5 countries with the most internet users?
  6. What is the correlation between internet usage (population share) and broadband subscriptions for 2019?




##  The data

#### DATA ([source](https://ourworldindata.org/internet)):

#### internet
- "Entity" - The name of the country, region, or group.
- "Code" - Unique id for the country (null for other entities).
- "Year" - Year from 1990 to 2019.
- "Internet_usage" -  The share of the entity's population who have used the internet in the last three months.

#### people
- "Entity" - The name of the country, region, or group.
- "Code" - Unique id for the country (null for other entities).
- "Year" - Year from 1990 to 2020.
- "Users" - The number of people who have used the internet in the last three months for that country, region, or group.

#### broadband
- "Entity" - The name of the country, region, or group.
- "Code" - Unique id for the country (null for other entities).
- "Year" - Year from 1998 to 2020.
- "Broadband_Subscriptions" - The number of fixed subscriptions to high-speed internet at downstream speeds >= 256 kbit/s for that country, region, or group.



### Import lib

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from pandasql import sqldf
import pandas as pd
import numpy as np


### Read & Investigate Data

In [ ]:
internet = pd.read_csv('data/internet.csv')

internet.head(2)

In [ ]:
internet.info()
## nulls in Code 

In [ ]:
people = pd.read_csv('data/people.csv')
people.head(2)

In [ ]:
people.info()
## nulls in code

In [ ]:
broadband = pd.read_csv('data/broadband.csv')
broadband.head(2) 


In [ ]:
broadband.info() 
## nulls in code

In [ ]:
#this table is related to the nulls in Code it has the groub of each country 
EntityGroup=pd.read_excel('EntityGroup.xlsx',sheet_name=1).rename(columns={'CountryCode':'Code'}).drop(columns=['CountryName','GroupCode']) 
EntityGroup.head(3)
 

In [ ]:
EntityGroup.info()

In [ ]:
EntityGroup[EntityGroup['Code']=='AFG']
# More than GroupName for same country 

### 1. What are the top 5 countries with the highest internet use (by population share) 1990-2019


In [ ]:
# the nulls is country Group
internet[internet['Code'].isna()]


In [ ]:
clean_internet=internet.dropna()
clean_internet.info()

In [ ]:
clean_internet.head(2)

In [ ]:
#BY Python
clean_internet.groupby("Entity")["Internet_Usage"].mean().sort_values(ascending=False).head(5)

In [ ]:
#BY SQL
Query1 = '''SELECT Entity,AVG(Internet_Usage) AS AVG_Internet_Usage
        FROM internet
        WHERE Entity NOT NULL
        GROUP BY Entity
        ORDER BY AVG(Internet_Usage) DESC
        LIMIT 5
     '''
q1=sqldf(Query1, globals())
q1

In [ ]:
fig = px.bar(q1, x="Entity", y="AVG_Internet_Usage", color='Entity',title = "Entity VS AVG_Internet_Usage From 1990-2019", width=800, height=400,text_auto='.3s')
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False,showlegend=False)

fig.show()

In [ ]:
## Kosovo, officially the Republic of Kosovo, is a partially recognised state in Southeast Europe. It lies at the centre of the Balkans.
## Curaçao, a Dutch Caribbean island, is known for its beaches tucked into coves and its expansive coral reefs rich with marine life. The capital

#### 1.1 What are the top 5 countries with the highest internet use (by population share) 2019


In [ ]:
#BY SQL
Query1_1 = '''SELECT Entity,Internet_Usage
        FROM internet
        WHERE Entity NOT NULL
        AND 
        Year=2019
        ORDER BY Internet_Usage DESC
        LIMIT 5
     '''
q1_1=sqldf(Query1_1, globals())
q1_1

### 2. How many people had internet access in those countries in 2019?


In [ ]:
top5=list(q1.Entity.values)

In [ ]:
top5 

In [ ]:
#BY Python
people[people['Entity'].isin(top5) & (people['Year']==2019) ].sort_values(by='Users', ascending=False)[['Entity','Users']]

In [ ]:
#BY SQL
Query2 = '''SELECT Entity,Users
        FROM people
        WHERE Entity  IN (SELECT Entity
                                 FROM internet
                                 WHERE Entity NOT NULL
                                                       
                                 GROUP BY Entity
                                 ORDER BY AVG(Internet_Usage) DESC
                                 LIMIT 5
                         )
        AND Year=2019
        ORDER BY Users DESC 
          
     '''
q2=sqldf(Query2, globals())
q2

In [ ]:
#BY SQL
Query2_1 = '''SELECT Entity,Users
        FROM people
        WHERE Entity IN('Kosovo', 'Curacao', 'Iceland', 'Norway', 'Sweden')
        AND Year=2019
        ORDER BY Users DESC 

     '''
q2_1=sqldf(Query2_1, globals())
q2_1


In [ ]:
fig=px.pie(q2,values='Users',names='Entity',hole=.5,hover_data=['Users'], width=800, height=400)
fig.update_traces(textposition='outside', textinfo='percent+label',pull=[0, 0.2, 0,0,0,0,0])
fig.update_layout(
    title_text="Number of users in countries with highest internet use  ",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='2019', x=0.50, y=0.5, font_size=20, showarrow=False),
                 ])
fig.show()

#### 2.1 How many people had internet access in those countries(['Bahrain', 'Qatar', 'Kuwait', 'United Arab Emirates', 'Denmark'] ) in 2019?


In [ ]:
#BY SQL
Query2_2 = '''SELECT Entity,Users
        FROM people
        WHERE Entity  IN (SELECT Entity
                                 FROM internet
                                 WHERE Code NOT NULL
                                AND 
                                 Year=2019
                                 
                                 ORDER BY Internet_Usage DESC
                                 LIMIT 5
                         )
        AND Year=2019
        ORDER BY Users DESC 
          
     '''
q2_2=sqldf(Query2_2, globals())
q2_2

In [ ]:
fig=px.pie(q2_2,values='Users',names='Entity',hole=.5,hover_data=['Users'], width=900, height=400)
fig.update_traces(textposition='outside', textinfo='percent+label',pull=[0, 0.2, 0,0,0,0,0])
fig.update_layout(
    title_text="Number of users in countries with highest internet use  ",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='2019', x=0.50, y=0.5, font_size=20, showarrow=False),
                 ])
fig.show()

### 3. What are the top 5 countries with the highest internet use for each of the following regions:
- 'Middle East & North Africa'
- 'Latin America & Caribbean'
- 'East Asia & Pacific'
- 'South Asia'
- 'North America'
- 'Europe & Central Asia'


In [ ]:
# contain all information about Internet_Usage , country, country group
joined=internet.merge(EntityGroup)

In [ ]:
joined.head()

#### 3.1 Top 5 in Middle East & North Africa

In [ ]:
#BY Python
joined[joined['GroupName']=='Middle East & North Africa'].groupby('Entity')['Internet_Usage'].mean().sort_values(ascending=False).head(5)


In [ ]:
#BY SQL
Query3_1 = '''SELECT Entity
        FROM internet LEFT JOIN EntityGroup ON internet.Code=EntityGroup.Code
        WHERE EntityGroup.GroupName='Middle East & North Africa'
        GROUP BY Entity 
        ORDER BY AVG(Internet_Usage) DESC 
        LIMIT 5
     '''
q3_1=sqldf(Query3_1, globals())
q3_1 


#### 3.2 Top 5 in Latin America & Caribbean

In [ ]:
#BY Python
joined[joined['GroupName']=='Latin America & Caribbean'].groupby('Entity')['Internet_Usage'].mean().sort_values(ascending=False).head(5)


In [ ]:
#BY SQL
Query3_2 = '''SELECT Entity
        FROM internet LEFT JOIN EntityGroup ON internet.Code=EntityGroup.Code
        WHERE EntityGroup.GroupName='Latin America & Caribbean'

        GROUP BY Entity 
        ORDER BY AVG(Internet_Usage) DESC 
        LIMIT 5
     '''
q3_2=sqldf(Query3_2, globals())
q3_2


#### 3.3 Top 5 in East Asia & Pacific

In [ ]:
#BY Python
joined[joined['GroupName']=='East Asia & Pacific'].groupby('Entity')['Internet_Usage'].mean().sort_values(ascending=False).head(5)


In [ ]:
#BY SQL
Query3_3 = '''SELECT Entity
        FROM internet LEFT JOIN EntityGroup ON internet.Code=EntityGroup.Code
        WHERE EntityGroup.GroupName='East Asia & Pacific'

        GROUP BY Entity 
        ORDER BY AVG(Internet_Usage) DESC 
        LIMIT 5
     '''
q3_3=sqldf(Query3_3, globals())
q3_3


#### 3.4 Top 5 in South Asia

In [ ]:
#BY Python
joined[joined['GroupName']=='South Asia'].groupby('Entity')['Internet_Usage'].mean().sort_values(ascending=False).head(5)


In [ ]:
#BY SQL
Query3_4 = '''SELECT Entity
        FROM internet LEFT JOIN EntityGroup ON internet.Code=EntityGroup.Code
        WHERE EntityGroup.GroupName='South Asia'
      
        GROUP BY Entity 
        ORDER BY AVG(Internet_Usage) DESC 
        LIMIT 5
     '''
q3_4=sqldf(Query3_4, globals())
q3_4


#### 3.5 Top 5 in North America

In [ ]:
#BY Python
joined[joined['GroupName']=='North America'].groupby('Entity')['Internet_Usage'].mean().sort_values(ascending=False).head(5)


In [ ]:
#there are only 3 in North America
joined[joined['GroupName']=='North America']['Entity'].unique()

In [ ]:
#BY SQL
Query3_5 = '''SELECT Entity
        FROM internet LEFT JOIN EntityGroup ON internet.Code=EntityGroup.Code
        WHERE EntityGroup.GroupName='North America'
        GROUP BY Entity 
        ORDER BY AVG(Internet_Usage) DESC 
        LIMIT 5
     '''
q3_5=sqldf(Query3_5, globals())
q3_5


#### 3.6 Top 5 in Europe & Central Asia

In [ ]:
#BY Python
joined[joined['GroupName']=='Europe & Central Asia'].groupby('Entity')['Internet_Usage'].mean().sort_values(ascending=False).head(5)


In [ ]:
#BY SQL
Query3_6 = '''SELECT Entity
        FROM internet LEFT JOIN EntityGroup ON internet.Code=EntityGroup.Code
        WHERE EntityGroup.GroupName='Europe & Central Asia'
        GROUP BY Entity 
        ORDER BY AVG(Internet_Usage) DESC 
        LIMIT 5
     '''
q3_6=sqldf(Query3_6, globals())
q3_6


### 4. Create a visualization for those five regions' internet usage over time.


In [ ]:
regions=[ 'Middle East & North Africa', 'Latin America & Caribbean', 'East Asia & Pacific', 'South Asia', 'North America', 'Europe & Central Asia']

In [ ]:
regions

In [ ]:
internetUsage_of_regions=internet[internet['Entity'].isin(regions)]
internetUsage_of_regions.head()

In [ ]:
fig = px.line(internetUsage_of_regions, x="Year", y="Internet_Usage", color='Entity',title = "Internet usage over time By Regions",width=1500, height=400)
fig.show()

In [ ]:
fig = px.bar(internetUsage_of_regions.query ('Year < 2018'), x="Entity", y="Internet_Usage", color="Entity",
  animation_frame="Year", animation_group="Entity", title = "Internet usage over time for seven regions from 1990 to 2017 for seven regions")



fig.show()

### 5. What are the 5 countries with the most internet users?


In [ ]:
people.info()

In [ ]:
people[people['Code'].isna()].head()


In [ ]:
# Group counteries in pepole table 
people[people['Code'].isna()]['Entity'].unique()

In [ ]:
clean_people=people.dropna()
clean_people.info()

In [ ]:
#there is one more group appear >> World
clean_people.query(' Year == 2020').sort_values(by='Users',ascending=False)[['Entity','Users']].head(5)

In [ ]:
clean_people[clean_people['Entity']=="World"].head(5)


In [ ]:
#By Python
clean_people.query('Entity !="World" & Year == 2020').sort_values(by='Users',ascending=False)[['Entity','Users']].head(5)

In [ ]:
#BY SQL
Query4 = '''SELECT Entity,Users
                FROM people 
                WHERE Entity NOT IN
                                (SELECT DISTINCT Entity
                                FROM people 
                                WHERE people.Code IS  NULL)
                                AND 
                Entity != "World" 
                                AND
                Year = 2020
                ORDER BY USERS DESC 
                LIMIT 5
     '''
q4=sqldf(Query4, globals())
q4


In [ ]:
#BY SQL
Query4_1 = '''SELECT Entity,Users
                FROM people 
                WHERE Code NOT NULL
                                AND 
                Entity != "World" 
                                AND
                Year = 2020
                ORDER BY Users DESC 
                LIMIT 5
     '''
q4_1=sqldf(Query4_1, globals())
q4_1


In [ ]:
fig=px.pie(q4_1,values='Users',names='Entity',hole=.5,hover_data=['Users'], width=900, height=400)
fig.update_traces(textposition='outside', textinfo='percent+label',pull=[0, 0.2, 0,0,0,0,0])
fig.update_layout(
    title_text=" Top 5 countries with the most internet users 2020",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='2020', x=0.50, y=0.5, font_size=20, showarrow=False),
                 ])
fig.show()

In [ ]:
fig = px.bar(q4_1, x="Entity", y="Users", color='Entity',title = "Top 5 countries with the most internet users 2020", width=800, height=400,text_auto='.4s')
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False,showlegend=False)

fig.show()

### 6. What is the correlation between internet usage (population share) and broadband subscriptions for 2019?


In [ ]:
broadband.info()

In [ ]:
broadband[broadband['Code'].isna()]

In [ ]:
clean_broadband=broadband.dropna()

In [ ]:
clean_broadband.info()

In [ ]:
clean_internet.info()

In [ ]:
clean_broadband19=clean_broadband.query('Year==2019')
clean_internet19=clean_internet.query('Year==2019')

In [ ]:
clean_broadband19.shape , clean_internet19.shape

In [ ]:
usage_subs=clean_internet19.merge(clean_broadband19)
usage_subs.head()

In [ ]:
usage_subs.shape

In [ ]:
usage_subs[['Internet_Usage','Broadband_Subscriptions']].corr() 

In [ ]:

sns.set_theme(color_codes=True)
g = sns.jointplot(x="Internet_Usage", y="Broadband_Subscriptions", data=usage_subs[['Internet_Usage','Broadband_Subscriptions']], ci=None,  kind="reg")


### Summary 

-  Top 5 countries with the highest internet use (by population share) 1990-2019 are :
   -  Kosovo
   -  Curacao
   -  Iceland
   -  Norway
   -  Sweden
-  Number of people had internet access in [Kosovo-Curacao-Iceland-Norway-Sweden]  in 2019 are 
   -  Sweden	9,702,513
   - 	Norway	5,241,320
   -	Iceland	357,179
   -  Kosovo   No Data
   -  Curacao  No Data
  

  
- Top 5 countries with the highest internet use (by population share) 2019 are :
  - Bahrain	
  - Qatar	
  - Kuwait	
  - United Arab Emirates	
  - Denmark	

-  Number of people had internet access in [Bahrain-Qatar-Kuwait-United Arab Emirates-Denmark]  in 2019 are 
     -  United Arab Emirates	9,133,361
     -  Denmark	            5,682,653
     -  Kuwait	               4,420,795
     -  Qatar	               2,797,495
     -  Bahrain	            1,489,735


-  the top 5 countries with the highest internet use for each of the following regions:
     - 'Middle East & North Africa'
       - United Arab Emirates    
       - Bahrain                 
       - Qatar                   
       - Malta                   
       - Israel(Occupation state)               
     
     - 'Latin America & Caribbean'
       - Curacao                  
       - Cayman Islands           
       - Barbados                 
       - Aruba                    
       - Saint Kitts and Nevis    

     - 'East Asia & Pacific'
       - South Korea    
       - New Zealand    
       - Japan          
       - Australia      
       - Singapore      
      
     - 'South Asia'
        - Maldives     
        - Bhutan        
        - Sri Lanka     
        - India         
        - Pakistan      
     
     - 'North America'
         - Bermuda          
         - Canada          
         - United States    
     
     - 'Europe & Central Asia'
        - Iceland        
        - Norway        
        - Sweden         
        - Denmark        
        - Netherlands   



- Highest internet use for All regions over time is :
  - 'North America'

- Lowest internet use for All regions over time is :
  - 'South Asia'

- Top  5 countries with the most internet users in 2020 :
  - China	 1,003,218,650
  - India	600,446,441
  - United States	305,371,298
  - Brazil	173,419,624
  - Indonesia	146,059,763

- There is a corr between 'Internet_Usage' & 'Broadband_Subscriptions' and it is = 0.557518

























